In [1]:
import pandas as pd
import numpy as np
import os
from tqdm import tqdm
from dotenv import dotenv_values

In [2]:
config = dotenv_values('.env')

In [3]:
test = pd.read_parquet(config["ENGINEERED_DATA"] + "test_fe.parquet")
test.head()

,customer_ID,P_2_mean,P_2_std,P_2_min,P_2_max,P_2_last,D_39_mean,D_39_std,D_39_min,D_39_max,...,D_63_nunique,D_64_count,D_64_last,D_64_nunique,D_66_count,D_66_last,D_66_nunique,D_68_count,D_68_last,D_68_nunique
0,00000469ba478561f23a92a868bd366de6f6527a684c9a...,0.601387,0.020190,0.568930,0.631315,0.568930,2.222222,3.527668,0,8,...,1,9,3,2,9,-1,1,9,6,2
1,00001bf2e77ff879fab36aa4fac689b9ba411dae63ae39...,0.862166,0.031436,0.794469,0.913501,0.841177,5.076923,6.034091,0,17,...,1,13,0,1,13,-1,1,13,6,1
2,0000210045da4f81e5f122c6bde5c2a617d03eef67f82c...,0.748955,0.061456,0.673112,0.835114,0.697522,6.000000,9.000000,0,23,...,1,13,3,2,13,1,1,13,4,2
3,00003b41e58ede33b8daf61ab56d9952f17c9ad1c3976c...,0.474728,0.028856,0.428457,0.514222,0.513186,15.846154,4.355957,7,23,...,1,13,2,1,13,-1,1,13,5,1
4,00004b22eaeeeb0ec976890c1d9bfc14fd9427e98c4ee9...,0.324100,0.049865,0.254478,0.425764,0.254478,11.846154,6.681394,1,26,...,1,13,2,2,13,-1,1,13,5,2


In [4]:
consensus = pd.read_csv(config["ENGINEERED_DATA"] + "test_labels.csv")
consensus.head()

,customer_ID,target
0,00000469ba478561f23a92a868bd366de6f6527a684c9a...,-127
1,00001bf2e77ff879fab36aa4fac689b9ba411dae63ae39...,0
2,0000210045da4f81e5f122c6bde5c2a617d03eef67f82c...,-127
3,00003b41e58ede33b8daf61ab56d9952f17c9ad1c3976c...,-127
4,00004b22eaeeeb0ec976890c1d9bfc14fd9427e98c4ee9...,1


In [5]:
test = test.set_index("customer_ID").join(consensus.set_index("customer_ID"))
test = test.reset_index()

In [6]:
new_data = test[test["target"] != -127]
new_data.head()

,customer_ID,P_2_mean,P_2_std,P_2_min,P_2_max,P_2_last,D_39_mean,D_39_std,D_39_min,D_39_max,...,D_64_count,D_64_last,D_64_nunique,D_66_count,D_66_last,D_66_nunique,D_68_count,D_68_last,D_68_nunique,target
1,00001bf2e77ff879fab36aa4fac689b9ba411dae63ae39...,0.862166,0.031436,0.794469,0.913501,0.841177,5.076923,6.034091,0,17,...,13,0,1,13,-1,1,13,6,1,0
4,00004b22eaeeeb0ec976890c1d9bfc14fd9427e98c4ee9...,0.324100,0.049865,0.254478,0.425764,0.254478,11.846154,6.681394,1,26,...,13,2,2,13,-1,1,13,5,2,1
5,00004ffe6e01e1b688170bbd108da8351bc4c316eacfef...,1.004837,0.003999,0.994927,1.009625,1.003475,2.230769,3.982333,0,14,...,13,0,1,13,-1,1,13,6,1,0
6,00007cfcce97abfa0b4fa0647986157281d01d3ab90de9...,0.452570,0.051298,0.351432,0.533795,0.351432,8.846154,8.234543,0,20,...,13,0,1,13,-1,1,13,5,2,1
9,0000b48a4f27dc1d61e78d081678e811620300b88eb3ab...,0.952516,0.012578,0.936205,0.989910,0.949838,0.153846,0.375534,0,1,...,13,0,1,13,-1,1,13,6,1,0


In [7]:
del test
train = pd.read_parquet(config["ENGINEERED_DATA"] + "train_fe.parquet")
train.head()

,customer_ID,P_2_mean,P_2_std,P_2_min,P_2_max,P_2_last,D_39_mean,D_39_std,D_39_min,D_39_max,...,D_64_count,D_64_last,D_64_nunique,D_66_count,D_66_last,D_66_nunique,D_68_count,D_68_last,D_68_nunique,target
0,0000099d6bd597052cdcda90ffabf56573fe9d7c79be5f...,0.933824,0.024194,0.868580,0.960384,0.934745,0.230769,0.832050,0,3,...,13,0,1,13,-1,1,13,6,1,0
1,00000fd6641609c6ece5454664794f0340ad84dddce9a2...,0.899820,0.022119,0.861109,0.929122,0.880519,7.153846,6.743468,0,19,...,13,0,1,13,-1,1,13,6,1,0
2,00001b22f846c82c51f6e3958ccd81970162bae8b007e8...,0.878454,0.028911,0.797670,0.904482,0.880875,0.000000,0.000000,0,0,...,13,2,1,13,-1,1,13,6,1,0
3,000041bdba6ecadd89a52d11886e8eaaec9325906c9723...,0.598969,0.020107,0.567442,0.623392,0.621776,1.538462,3.017046,0,9,...,13,0,1,13,-1,1,13,3,3,0
4,00007889e4fcd2614b6cbe7f8f3d2e5c728eca32d9eb8a...,0.891679,0.042325,0.805045,0.940382,0.871900,0.000000,0.000000,0,0,...,13,0,1,13,1,1,13,6,1,0


In [9]:
train = pd.concat([train, new_data], ignore_index=True)
train

,customer_ID,P_2_mean,P_2_std,P_2_min,P_2_max,P_2_last,D_39_mean,D_39_std,D_39_min,D_39_max,...,D_64_count,D_64_last,D_64_nunique,D_66_count,D_66_last,D_66_nunique,D_68_count,D_68_last,D_68_nunique,target
0,0000099d6bd597052cdcda90ffabf56573fe9d7c79be5f...,0.933824,0.024194,0.868580,0.960384,0.934745,0.230769,0.832050,0,3,...,13,0,1,13,-1,1,13,6,1,0
1,00000fd6641609c6ece5454664794f0340ad84dddce9a2...,0.899820,0.022119,0.861109,0.929122,0.880519,7.153846,6.743468,0,19,...,13,0,1,13,-1,1,13,6,1,0
2,00001b22f846c82c51f6e3958ccd81970162bae8b007e8...,0.878454,0.028911,0.797670,0.904482,0.880875,0.000000,0.000000,0,0,...,13,2,1,13,-1,1,13,6,1,0
3,000041bdba6ecadd89a52d11886e8eaaec9325906c9723...,0.598969,0.020107,0.567442,0.623392,0.621776,1.538462,3.017046,0,9,...,13,0,1,13,-1,1,13,3,3,0
4,00007889e4fcd2614b6cbe7f8f3d2e5c728eca32d9eb8a...,0.891679,0.042325,0.805045,0.940382,0.871900,0.000000,0.000000,0,0,...,13,0,1,13,1,1,13,6,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1055979,ffff1cf18b40bacfe82d222c6d214eb8400c5a82b207ff...,0.193755,0.138860,-0.002841,0.347165,-0.002841,6.111111,9.765301,0,27,...,9,2,2,9,-1,1,9,4,2,1
1055980,ffff6e21564cd8768222aef92c7a1b332532b8215f71b6...,0.840948,0.043475,0.768653,0.917870,0.827208,0.538462,1.330124,0,4,...,13,0,1,13,1,1,13,6,1,0
1055981,ffff824b313399b00db6bc930b83f1a2188d8b1dbd3a31...,0.917882,0.038348,0.855331,0.977334,0.977334,2.846154,5.785703,0,18,...,13,3,1,13,-1,1,13,6,1,0
1055982,ffff952c631f2c911b8a2a8ca56ea6e656309a83d2f64c...,0.631613,0.015759,0.597667,0.646915,0.646915,0.000000,0.000000,0,0,...,13,0,1,13,-1,1,13,6,1,0


In [10]:
train.sort_values(by='customer_ID', inplace=True)

In [11]:
train.to_parquet(config["ENGINEERED_DATA"] + "extended_train_fe.parquet", index=False)

In [12]:
del train
confirm = pd.read_parquet(config["ENGINEERED_DATA"] + "extended_train_fe.parquet")
confirm

,customer_ID,P_2_mean,P_2_std,P_2_min,P_2_max,P_2_last,D_39_mean,D_39_std,D_39_min,D_39_max,...,D_64_count,D_64_last,D_64_nunique,D_66_count,D_66_last,D_66_nunique,D_68_count,D_68_last,D_68_nunique,target
0,0000099d6bd597052cdcda90ffabf56573fe9d7c79be5f...,0.933824,0.024194,0.868580,0.960384,0.934745,0.230769,0.832050,0,3,...,13,0,1,13,-1,1,13,6,1,0
1,00000fd6641609c6ece5454664794f0340ad84dddce9a2...,0.899820,0.022119,0.861109,0.929122,0.880519,7.153846,6.743468,0,19,...,13,0,1,13,-1,1,13,6,1,0
2,00001b22f846c82c51f6e3958ccd81970162bae8b007e8...,0.878454,0.028911,0.797670,0.904482,0.880875,0.000000,0.000000,0,0,...,13,2,1,13,-1,1,13,6,1,0
3,00001bf2e77ff879fab36aa4fac689b9ba411dae63ae39...,0.862166,0.031436,0.794469,0.913501,0.841177,5.076923,6.034091,0,17,...,13,0,1,13,-1,1,13,6,1,0
4,000041bdba6ecadd89a52d11886e8eaaec9325906c9723...,0.598969,0.020107,0.567442,0.623392,0.621776,1.538462,3.017046,0,9,...,13,0,1,13,-1,1,13,3,3,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1055979,ffff952c631f2c911b8a2a8ca56ea6e656309a83d2f64c...,0.631613,0.015759,0.597667,0.646915,0.646915,0.000000,0.000000,0,0,...,13,0,1,13,-1,1,13,6,1,0
1055980,ffff9984b999fccb2b6127635ed0736dda94e544e67e02...,0.786838,0.014312,0.756983,0.802953,0.800522,7.384615,6.212064,0,18,...,13,3,1,13,-1,1,13,5,1,0
1055981,ffffa5c46bc8de74f5a4554e74e239c8dee6b9baf38814...,0.804454,0.037442,0.754129,0.856981,0.754129,0.923077,2.752621,0,10,...,13,3,2,13,-1,1,13,3,2,1
1055982,ffffcf5df59e5e0bba2a5ac4578a34e2b5aa64a1546cd3...,0.547049,0.053224,0.471303,0.634783,0.471303,1.769231,3.982333,0,14,...,13,0,1,13,-1,1,13,6,2,1


In [ ]:
import matplotlib.pyplot as plt
plt.pie(confirm['target'])